# Project Overview
This project builds an ETL (Extract, Transform, Load) pipeline for Formula 1 race data using the FastF1 Python library. The pipeline automates the collection of race results, cleans and validates the dataset, and loads it into a SQLite database for analysis. Finally, the project runs an example SQL query to identify the Top 10 drivers by points in a selected Grand Prix event.

## Section 1: Installing Dependencies
Using pip install fastf1 to install the FastF1 library, which provided a structured access to Formula 1 data.


In [3]:
pip install fastf1


Note: you may need to restart the kernel to use updated packages.


## Section 2: Configuring Data Caching
To reduce repeated API requesnts and speedinf up the ETL pipleline, a local cache_folder was created.


In [12]:

import os
import fastf1

# Create folder if it doesn't exist
if not os.path.exists('cache_folder'):
    os.makedirs('cache_folder')

fastf1.Cache.enable_cache('cache_folder')

## Section 3: Building the ETL Pipeline
In this step, the ETL pipeline was implemented. The pipeline performed the following tasks:
Extract: Fetched Formula 1 race data using the FastF1 API.
Transform: Cleaned and standardized the data.
Load: Saved the cleaned data into a local SQLite database for structured storage.
Analyzed: Queried the data for top 10 drivers by points.

In [36]:
import fastf1
import pandas as pd
import sqlite3
import os

# Enable caching 
if not os.path.exists("f1_cache"):
    os.makedirs("f1_cache")
fastf1.Cache.enable_cache("f1_cache")


# Simple Cleaning Functions

def drop_nulls(df, columns):
    """Remove rows with missing values in selected columns."""
    return df.dropna(subset=columns)

def convert_types(df, col_types):
    """Convert columns to a specific type (like int or float)."""
    for col, dtype in col_types.items():
        if col in df.columns:
            df[col] = df[col].astype(dtype, errors="ignore")
    return df

def strip_whitespace(df, columns):
    """Remove spaces before/after names or text."""
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    return df

def validate_data(df):
    """Quick checks on the data."""
    if "points" in df.columns:
        negatives = (df["points"] < 0).sum()
        if negatives > 0:
            print(f"⚠ Found {negatives} negative points!")
    print("Validation complete")


# ETL Pipeline

def etl_pipeline(year=2025, grand_prix="Silverstone", session_type="R"):
    print(f"Starting ETL for {grand_prix} {year}")

    # Extract
    session = fastf1.get_session(year, grand_prix, session_type)
    session.load(laps=False, telemetry=False)
    df = session.results.copy()

    # Transform
    df.columns = [c.lower() for c in df.columns]
    df = df.rename(columns={
        "broadcastname": "driver",
        "teamname": "team"
    })

    df = drop_nulls(df, ["driver", "points"])
    df = convert_types(df, {"points": "float", "position": "int"})
    df = strip_whitespace(df, ["driver", "team"])
    validate_data(df)

    # Load
    conn = sqlite3.connect("f1_results.db")
    table_name = f"{grand_prix.lower()}_{year}"
    df.to_sql(table_name, conn, if_exists="replace", index=False)

    # Query
    query = f"""
        SELECT driver, SUM(points) as total_points
        FROM {table_name}
        GROUP BY driver
        ORDER BY total_points DESC
        LIMIT 10
    """
    top_drivers = pd.read_sql(query, conn)

    conn.close()
    print("ETL complete!")

    return df, top_drivers


## Section 4: Running the pipleine


In [42]:

if __name__ == "__main__":
    df_clean, top_drivers = etl_pipeline(year=2025, grand_prix="Silverstone")
    print(top_drivers)
    df_clean.to_csv("f1_british_gp_2025_cleaned.csv", index=False)


core           INFO 	Loading data for British Grand Prix - Race [v3.6.0]
2025-08-29 19:36:57,056 - INFO - Loading data for British Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
2025-08-29 19:36:57,059 - INFO - Using cached data for session_info
req            INFO 	Using cached data for driver_info
2025-08-29 19:36:57,060 - INFO - Using cached data for driver_info
req            INFO 	Using cached data for weather_data
2025-08-29 19:36:57,079 - INFO - Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
2025-08-29 19:36:57,081 - INFO - Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', '22', '12', '6', '5', '30', '43']
2025-08-29 19:36:57,083 - INFO - Finished loading data for 20 drivers: ['4', '81', '27', '44', '1', '10', '18', '23', '14', '63', '87', '55', '31', '16', 

Starting ETL for Silverstone 2025
Validation complete
ETL complete!
         driver  total_points
0      L NORRIS          25.0
1     O PIASTRI          18.0
2  N HULKENBERG          15.0
3    L HAMILTON          12.0
4  M VERSTAPPEN          10.0
5       P GASLY           8.0
6      L STROLL           6.0
7       A ALBON           4.0
8      F ALONSO           2.0
9     G RUSSELL           1.0


/var/folders/_8/gzplx5cs0156hr34r3l9wzmc0000gn/T/ipykernel_10903/3871072654.py:66: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df.to_sql(table_name, conn, if_exists="replace", index=False)


## Section 5: Analyzing the data
After loading the  data into a SQLite database, a simple SQL query was ran to find the top 10 drivers by total points in the race.

In [46]:
# Connect to SQLite database 
conn = sqlite3.connect("f1_results.db")

# Run SQL query
query = """
SELECT driver, SUM(points) AS total_points
FROM silverstone_2025
GROUP BY driver
ORDER BY total_points DESC
LIMIT 10;
"""

top_drivers = pd.read_sql_query(query, conn)
print(top_drivers)

conn.close()


         driver  total_points
0      L NORRIS          25.0
1     O PIASTRI          18.0
2  N HULKENBERG          15.0
3    L HAMILTON          12.0
4  M VERSTAPPEN          10.0
5       P GASLY           8.0
6      L STROLL           6.0
7       A ALBON           4.0
8      F ALONSO           2.0
9     G RUSSELL           1.0
